
## 引入库

In [1]:
import pyautogui
# -*-  coding: utf-8 -*-
# Author: cw
# Datetime : 2020
# software: PyCharm
# 收获：
from selenium import webdriver
from selenium.webdriver import FirefoxProfile
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import math
import time
import logging
#from datetime import datetime

## 很多小工具

In [41]:
# 下载文书
def download(index='0'):
    try:
        quanxuan = browser.find_element_by_xpath('/html/body/div/div[4]/div[2]//div[2]/div[4]/a[1]')
        # browser.execute_script("arguments[0].style.visibility='hidden'", quanxuan)
        element = browser.find_element_by_xpath("//div[@class='List_label clearfix']")
        browser.execute_script("arguments[0].style.visibility='hidden'", element)
        element = browser.find_element_by_xpath("//div[@class='LM_list']")
        browser.execute_script("arguments[0].style.visibility='hidden'", element)

        quanxuan.click()
        time.sleep(5)


        download = browser.find_element_by_xpath('/html/body/div/div[4]/div[2]//div[2]/div[4]/a[3]')
        download.click()
    except:
        print('无法下载',get_rules(),index)
      
    
# 循环删除某个条件
def delete_rule_each(rule_text):
    search_rule = browser.find_element_by_xpath("/html/body/div/div[4]/div[2]//div/div/div[2]/div[2]")
    #for rule in search_rule.find_elements_by_tag_name('p'):
    rules = search_rule.find_elements_by_tag_name('p')
    for del_rule in rules:
        if rule_text in del_rule.text:
            print('删除',del_rule.text)
            del_rule.find_element_by_tag_name('i').click()
            time.sleep(5)
            return
        
    print('没有找到那一条',rule_text,[rule.text for rule in rules])

# 删除最后一个条件
def delete_rule(rule_text):
    search_rule = browser.find_element_by_xpath("/html/body/div/div[4]/div[2]//div/div/div[2]/div[2]")
    #for rule in search_rule.find_elements_by_tag_name('p'):
    rules = search_rule.find_elements_by_tag_name('p')
    del_rule = rules[-1]
    if rule_text in del_rule.text:
        print('删除',del_rule.text)
        del_rule.find_element_by_tag_name('i').click()
        time.sleep(5)
    else:
        print('出现未知错误，不能删掉这一条',rule_text,del_rule.text)
        

def get_rules():
    search_rule = browser.find_element_by_xpath("/html/body/div/div[4]/div[2]//div/div/div[2]/div[2]")
    rules = search_rule.find_elements_by_tag_name('p')
    text_list = [rule.text.split('：')[1] for rule in rules]
    return text_list


# 获得左边目录树
def get_left_tree():
    # 0: 关键字 1: 案由 2: 法院层级 3:地域及法院 4: 裁判年份 5: 审判程序 6:文书类型 7:案例等级
    left_tree = browser.find_elements_by_xpath('/html/body/div/div[4]/div[1]//div/div[2]/ul') 
    return left_tree

# 获取当前文书的数量
def check_results_len():
    result = browser.find_element_by_xpath("/html/body/div/div[4]/div[2]//div[1]/div[2]/span")
    return result.text

xpath = '/html/body/div/div[4]/div[1]//div/div[2]/ul'

# 只获取文本的第一段
def flit_text(text):
    return text.split('\n')[0]

# 点开新的链接后，得到当前的树
def each_time_find_tree(deep_list,tree,xpath):
#     print(deep_list,flit_text(tree.text))
    if deep_list == []:
        return tree,xpath
    else:
        xpath+=f'/ul/li[{deep_list[0]+1}]'
#         print(deep_list,xpath)
        tree = get_anyou_from_lefttree(xpath)
        return each_time_find_tree(deep_list[1:],tree,xpath)
    
# 得到案由
def get_anyou_from_lefttree(xpath):
    anyou = browser.find_elements_by_xpath('/html/body/div/div[4]/div[1]//div/div[2]/ul')[1]
    return anyou.find_element_by_xpath(xpath)

def show_15():
    wait = WebDriverWait(browser,10)
    time.sleep(1)
    from selenium.webdriver.support.select import Select
    selector =Select(browser.find_element_by_xpath("/html/body/div/div[4]/div[2]//div[8]/div/select"))
    selector.select_by_index(2)
    # 直到第15个元素出现
    wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div/div[4]/div[2]//div[17]')))
    # wait.until(len(browser.find_elements_by_xpath("/html/body/div/div[4]/div[2]//div[@class='LM_list']")) == 15)

# 点击的path的操作
def click_xpath(xpath):
    button = browser.find_element_by_xpath(xpath)
    button.click() 

## 递归相关全部代码

In [56]:
# 按照年份搜索 指定年份
def search_by_year_special(year_name):
    left_tree = get_left_tree()
    years = get_left_tree()[4].find_elements_by_tag_name('li')
    print([each.text for each in years])
    for year in years:
        if year_name == year.text.split('(')[0]:
            if year.find_elements_by_tag_name('a') and year.find_element_by_tag_name('a').is_displayed():
                year.find_element_by_tag_name('a').click()
                time.sleep(10)
                print('成功选择条件',year_name)
                search_by_type()
            else:
                print(year_name,'爬不了')
            return
        



# 按照年份搜索（第一层）
def search_by_year():
    left_tree = get_left_tree()
    for i in range(len(left_tree[4].find_elements_by_tag_name('li'))):
        years = get_left_tree()[4].find_elements_by_tag_name('li')
        print('当前的年份',[year.text for year in years])
        if i < len(years) and years[i].find_elements_by_tag_name('a') and years[i].find_element_by_tag_name('a').is_displayed():
            years[i].find_element_by_tag_name('a').click()
            time.sleep(5)
            # 
            if int(check_results_len()) <= 615:
                print(get_rules(),'当前条件小于600条，开始爬取',int(check_results_len()))
                crwal()
                time.sleep(5)
            # 第二层 按照案件类型搜索
            else:
                print(get_rules(),'当前条件大于600条，调用文书类型递归',int(check_results_len()))
                search_by_type()
                time.sleep(5)
            delete_rule('裁判年份')
        
# 根据文书类型选择（第二层）
def search_by_type():
    left_tree = get_left_tree()
    for i in range(len(left_tree[6].find_elements_by_tag_name('li'))):
        text_type = get_left_tree()[6].find_elements_by_tag_name('li')
        print('当前的type',[year.text for year in text_type])
        if i < len(text_type) and text_type[i].find_elements_by_tag_name('a') and text_type[i].find_element_by_tag_name('a').is_displayed():
            text_type[i].find_element_by_tag_name('a').click()
            time.sleep(5)
            if int(check_results_len()) <= 615:
                print(get_rules(),'当前条件小于600条，开始爬取',int(check_results_len()))
                crwal()
                time.sleep(5)
            else:
                print(get_rules(),'当前条件大于600条，调用案由递归',int(check_results_len()))
                search_by_case_tree()
                time.sleep(5)
            delete_rule('文书类型')
        
# 使用深度遍历的方法得到树
def dfs_get_tree(deep_list,tree,deep,xpath):
    if deep == len(deep_list):
        return tree
    else:
        #print('当前的tree节点',flit_text(tree.text))
#         tree = get_anyou_from_lefttree(xpath)
        #print([each.text for each in  tree])
        #tree = tree[deep_list[deep]]
        print(deep,deep_list[deep],flit_text(tree.text))
        # 如果已经点开了
        if tree.find_elements_by_xpath('./ul'):
            print('已经点开了',flit_text(tree.text))
            # 进入到案由模块
            left_tree = get_left_tree()[1]
            # 得到现在的树节点
            tree,_ = each_time_find_tree(deep_list[:deep+1],left_tree,xpath)
#             print('now in ',flit_text(tree.text))
            return dfs_get_tree(deep_list,tree,deep+1,xpath)
        else:
            print('还没点开',flit_text(tree.text))
            if tree.find_elements_by_tag_name('i') and tree.find_elements_by_tag_name('i')[0].is_displayed():
                # 点开加号并且重新获取tree
                tree.find_elements_by_tag_name('i')[0].click()
                time.sleep(5)
                # 进入到案由模块
                left_tree = get_left_tree()[1]
                # 得到现在的树节点
                # 因为传进来的deep_list少了一位，所以判断的deep要加一位
                tree,_ = each_time_find_tree(deep_list[:deep+1],left_tree,xpath)
    #             print('now in ',flit_text(tree.text))
                return dfs_get_tree(deep_list,tree,deep+1,xpath)
            else:
                print('点不开',tree.text.split())
                return tree


def get_now_tree(deep_list):
    # 进入到案由模块
    left_tree = get_left_tree()[1]
    # 如果是空的，那就直接返回案由模块
    if deep_list == []:
        return left_tree
    else:
        # 第一层直接获得对应的值
        left_tree = left_tree.find_elements_by_tag_name('li')
        #print(left_tree[0].text)
                 #/html/body/div/div[4]/div[1]/d/div/div[2] 
            
        xpath = f'./li[{deep_list[0]+1}]'
        # xpath会匹配很多，只有第二个才是
        tree = get_anyou_from_lefttree(xpath)
        if 'jstree-closed' in tree.get_attribute('class'):
            if tree.find_elements_by_tag_name('i') and tree.find_elements_by_tag_name('i')[0].is_displayed():
                # 点开加号并且重新获取tree
                tree.find_elements_by_tag_name('i')[0].click()
                time.sleep(10)
                tree = get_now_tree(deep_list)
                print('有加号点一下',flit_text(tree.text))
                
                
        return dfs_get_tree(deep_list[1:], tree,0,xpath)
    
import re
def search_by_list(deep_list):
    
    tree = get_now_tree(deep_list)
    print('1 get_now_tree的位置',deep_list,tree.text.split('\n'),tree.get_attribute('class'),'有无列表：',len(tree.find_elements_by_tag_name('ul')))

    # 如果当前有加号
    if 'jstree-closed' in tree.get_attribute('class'):
        if tree.find_elements_by_tag_name('i') and tree.find_elements_by_tag_name('i')[0].is_displayed():
            # 点开加号并且重新获取tree
            print('有加号点一下',flit_text(tree.text))
            tree.find_elements_by_tag_name('i')[0].click()
            time.sleep(10)
            tree = get_now_tree(deep_list)
            #print('2 当前树的位置',deep_list,flit_text(tree.text),tree.get_attribute('class'),tree.find_elements_by_tag_name('ul'))
            print('2 get_now_tree的位置',deep_list,flit_text(tree.text),tree.get_attribute('class'),'有无列表：',len(tree.find_elements_by_tag_name('ul')))
            
        
        
    number = re.findall("\((\d+)\)",tree.text)[0]
    # 如能够找到子树 且数量大于600
    if len(tree.find_elements_by_tag_name('ul')) and int(number) > 600:
        #print('could find son',deep_list,flit_text(tree.text))
        
        case_list = tree.find_element_by_tag_name('ul').find_elements_by_tag_name('li')
        case_list_len = len(case_list)
        #print('长度为',case_list_len)
        for i in range(case_list_len):
            # 刷新页面后要重新获取一次
            root_tree = get_now_tree(deep_list)
            #print('3 当前树的位置',deep_list,flit_text(root_tree.text),root_tree.get_attribute('class'),root_tree.find_elements_by_tag_name('ul'))
            case_list = root_tree.find_element_by_tag_name('ul').find_elements_by_tag_name('li')
            # 获取当前的案件名称
            case = case_list[i]

            print([each.text for each in  case_list],case.text)
            
            number = re.findall("\((\d+)\)",case.text)[0]
            if int(number) <= 615:
                print('1 当前选择的文书类型为',flit_text(case.text))
                if case.find_elements_by_tag_name('a') and case.find_elements_by_tag_name('a')[0].is_displayed():
                        case.find_elements_by_tag_name('a')[0].click()
                        print('开始爬取')
                        crwal()
                        time.sleep(5)
                        delete_rule('案由')
            
            else:
                
                # 有一种情况是，点开加号后，里面没东西
                if 'jstree-closed' in case.get_attribute('class'):
                    #print('有加号点一下',flit_text(case.text))
                    if case.find_elements_by_tag_name('i') and case.find_elements_by_tag_name('i')[0].is_displayed():
                        case.find_elements_by_tag_name('i')[0].click()
                        time.sleep(10)
                        # 点开加号并且重新获取tree
                        root_tree = get_now_tree(deep_list)
                        #print('3 当前树的位置',deep_list,flit_text(root_tree.text),root_tree.get_attribute('class'),root_tree.find_elements_by_tag_name('ul'))
                        case_list = root_tree.find_element_by_tag_name('ul').find_elements_by_tag_name('li')
                        # 获取当前的案件名称
                        case = case_list[i]
                    
                # 检查子树
                plus = 'jstree-open' in case.get_attribute('class')
                    
                # 如果有子树
                if plus:
                    print('有子树',i,flit_text(case.text))
                    print('2 当前选择的文书类型为',flit_text(case.text), '需要增加深度')
                    # 添加一次深度
                    deep_list_add = deep_list.copy()
                    deep_list_add.append(i)
                    search_by_list(deep_list_add)
                else:
                    print('无子树',i,flit_text(case.text))
                    print('3 当前选择的文书类型为',flit_text(case.text))
                    # 直接选中当前标签的话需要用这个
                    if case.find_elements_by_tag_name('a') and case.find_element_by_tag_name('a').is_displayed():
                        case.find_element_by_tag_name('a').click()
                        print('开始爬取')
                        crwal()
                        time.sleep(5)
                        delete_rule('案由')
    else:
        print('没有子节点或者当前数字小于600',tree.text.split('\n'),number)
        print('4 当前选择的文书类型为',flit_text(tree.text))
        # 直接选中当前标签的话需要用这个
        if tree.find_elements_by_tag_name('a'):
            if tree.find_elements_by_tag_name('a') and tree.find_element_by_tag_name('a').is_displayed():
                tree.find_element_by_tag_name('a').click()
                print('开始爬取')
                crwal()
                time.sleep(5)
                delete_rule('案由')
    

# 深度和要访问的下标
def search_by_case_tree(deep_list = []):
    print('当前要爬取的',get_rules(),deep_list)
    try: 
        if deep_list == []:
            root_tree = get_now_tree(deep_list)
            root_len = len(root_tree.find_elements_by_tag_name('li'))
            for i in range(root_len):
    #             root_tree = get_now_tree(deep_list)
    #             tree = root_tree.find_elements_by_tag_name('li')[i]
                root_tree = get_now_tree(deep_list)
                print('第一个节点',flit_text(root_tree.find_elements_by_tag_name('li')[i].text),i)
                return_tree = search_by_list([i])
        else:
            search_by_list(deep_list)
    except:
        import traceback

        print('error 出错了',get_rules(),deep_list,traceback.format_exc())
        
        
# 爬取每一页的内容
def crwal():
    try:
        show_15()
    except:
        pass
        
    for i in range(int(600/15)):
        page_div = browser.find_elements_by_xpath('//*[@id="_view_1545184311000"]/div')
        
        rules = get_rules()
        rules.append(str(i))
        print(rules)
        save_name = '_'.join(rules)
        
        
        download(str(i))
        if len(page_div)!=18:
            print('已经到达末尾页')
            return
        page_div = browser.find_elements_by_xpath('//*[@id="_view_1545184311000"]/div')
        pages = page_div[-1].find_elements_by_tag_name('a')
        
        # 点击下一页
        next_page = pages[-2].text
        #print('下一页的元素',next_page)
        try:
            pages[-1].click()
        except:
            print('已经到达末尾页')
            return
        

        # 直到下一页出现
    #     wait.until(EC.presence_of_element_located((By.XPATH, f'/html/body/div/div[4]/div[2]//div[18]/a[{next_page}]')))
        time.sleep(5)


## 初始化+登录

In [53]:
# 无图
profile = FirefoxProfile()
profile.set_preference('browser.migration.version', 9001)
profile.set_preference('permissions.default.image', 2)


profile.set_preference('browser.helperApps.neverAsk.saveToDisk','application/octet-stream')
profile.set_preference('browser.download.dir', 'E:\判决文书\judment_learning\中国裁判文书网\save')


from selenium.webdriver.common.proxy import Proxy, ProxyType
 
 

browser = webdriver.Firefox(firefox_profile=profile)


"""网页获取"""
browser.get('http://wenshu.court.gov.cn')
time.sleep(5)
login = browser.find_element_by_xpath('/html/body/div[1]/div[2]/ul/li[1]/a')
login.click()
wait = WebDriverWait(browser,10)
time.sleep(5)
browser.switch_to.frame('contentIframe')

phone = browser.find_element_by_xpath('/html/body/div/div/form/div[1]/div[1]/div/div/div/input') 
phone.clear()
phone.send_keys('用户名')
time.sleep(1)

pwd = browser.find_element_by_xpath('/html/body/div/div/form/div[1]/div[2]/div/div/div/input') 
pwd.clear()
pwd.send_keys('密码')
time.sleep(1)

login = browser.find_element_by_xpath('/html/body/div/div/form/div[3]/span')
login.click()
time.sleep(5)

## 法院名字

In [54]:
city_list = ['武汉市新洲区人民法院', '武汉市洪山区人民法院', 
             '武汉市江夏区人民法院', '武汉市青山区人民法院 ',
            '武汉市硚口区人民法院','武汉市江汉区人民法院','武汉市武昌区人民法院',
            '武汉市东西湖区人民法院','武汉市江岸区人民法院','武汉市汉阳区人民法院',
             '武汉市汉南区人民法院','武汉经济技术开发区人民法院','武汉市黄陂区人民法院','武汉市蔡甸区人民法院']

## 按照年份爬取

In [ ]:
for city in city_list:
    print('当前爬取',city)
    num=browser.window_handles
    print(num)
    browser.switch_to.window(num[0])
    
    try:
        delete_rule_each('裁判年份')
        time.sleep(3)
        delete_rule_each('文书类型')
        time.sleep(3)
        delete_rule_each('案由')
        time.sleep(3)
    except:
        pass

    
    # 点击高级搜索
    adv_search = browser.find_element_by_xpath("/html/body/div/div[3]//div/div[1]/div[1]")
    adv_search.click()
    time.sleep(2)

    name = city
    name_input = browser.find_element_by_xpath('//*[@id="s2"]')
    name_input.clear()
    for each in name:
        name_input.send_keys(each)
        time.sleep(0.25)


    # 选择文书公开
    # 点击公开类型
    browser.find_element_by_xpath("/html/body/div/div[3]//div/div[3]/div[1]/div[12]/div/div/div").click()
    time.sleep(2)
    # 选择文书公开
    browser.find_element_by_xpath("/html/body/div/div[3]//div/div[3]/div[1]/div[12]/div/div/ul/li[2]").click()
    time.sleep(2)


    search = browser.find_element_by_xpath('//*[@id="searchBtn"]')
    search.click() 
    time.sleep(5)

    search_by_year_special('2020')

当前爬取 武汉市新洲区人民法院
['15']
没有找到那一条 裁判年份 ['法院名称：武汉市新洲区人民法院', '公开类型：文书公开']
没有找到那一条 文书类型 ['法院名称：武汉市新洲区人民法院', '公开类型：文书公开']
没有找到那一条 案由 ['法院名称：武汉市新洲区人民法院', '公开类型：文书公开']
['2021(119)', '2020(5741)', '2019(5310)', '2018(4422)', '2017(4122)', '2016(3343)', '2015(2593)', '2014(2927)', '2013(37)', '2012(2)', '2010(1)', '2009(1)']
成功选择条件 2020
当前的type ['判决书(2488)', '裁定书(3246)', '调解书(2)', '决定书(5)']
['武汉市新洲区人民法院', '文书公开', '2020', '判决书'] 当前条件大于600条，调用案由递归 2488
当前要爬取的 ['武汉市新洲区人民法院', '文书公开', '2020', '判决书'] []
第一个节点 刑事案由(586) 0
有加号点一下 刑事案由(586)
1 get_now_tree的位置 [0] ['刑事案由(586)', '危害公共安全罪(322)', '破坏社会主义市场经济秩序罪(1)', '侵犯公民人身权利、民主权利罪(58)', '侵犯财产罪(105)', '妨害社会管理秩序罪(98)', '贪污贿赂罪(9)'] jstree-node jstree-open 有无列表： 1
没有子节点或者当前数字小于600 ['刑事案由(586)', '危害公共安全罪(322)', '破坏社会主义市场经济秩序罪(1)', '侵犯公民人身权利、民主权利罪(58)', '侵犯财产罪(105)', '妨害社会管理秩序罪(98)', '贪污贿赂罪(9)'] 586
4 当前选择的文书类型为 刑事案由(586)
开始爬取
['武汉市新洲区人民法院', '文书公开', '2020', '判决书', '刑事案由', '0']
['武汉市新洲区人民法院', '文书公开', '2020', '判决书', '刑事案由', '1']
['武汉市新洲区人民法院', '文书公开', '2020', '判决书', 

## 全量爬取

In [ ]:
for city in city_list:
    num=browser.window_handles
    print(num)
    browser.switch_to.window(num[0])
    
    try:
        delete_rule_each('裁判年份')
        time.sleep(3)
        delete_rule_each('文书类型')
        time.sleep(3)
        delete_rule_each('案由')
        time.sleep(3)
    except:
        pass

    
    # 点击高级搜索
    adv_search = browser.find_element_by_xpath("/html/body/div/div[3]//div/div[1]/div[1]")
    adv_search.click()
    time.sleep(2)

    name = city
    name_input = browser.find_element_by_xpath('//*[@id="s2"]')
    name_input.clear()
    for each in name:
        name_input.send_keys(each)
        time.sleep(0.25)


    # 选择文书公开
    # 点击公开类型
    browser.find_element_by_xpath("/html/body/div/div[3]//div/div[3]/div[1]/div[12]/div/div/div").click()
    time.sleep(2)
    # 选择文书公开
    browser.find_element_by_xpath("/html/body/div/div[3]//div/div[3]/div[1]/div[12]/div/div/ul/li[2]").click()
    time.sleep(2)


    search = browser.find_element_by_xpath('//*[@id="searchBtn"]')
    search.click() 
    time.sleep(5)

    search_by_year()
